<a href="https://colab.research.google.com/github/navjotsingh151/Sensor-TimeSeriesAnalysis/blob/master/TimeSeriesAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Time Series analysis
Data  is collected from sensor A at **3 seconds interval** i.e. 480 cycles per day. Data is gathered for a week time frame 

# Data Cleaning

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

In [167]:
!pip install chart_studio

     |████████████████████████████████| 71kB 2.2MB/s 


In [0]:
data_a = pd.read_csv("https://raw.githubusercontent.com/navjotsingh151/Sensor-TimeSeriesAnalysis/master/Data/Data_A.csv")
data_a.head()
data_a.rename(columns={"Timestamp ":"Timestamp"}, inplace=True)

In [153]:
data_a = data_a.sort_values(["Timestamp"], ascending = (True))
data_a.reset_index(inplace=True)
data_a.drop('index', axis = 1, inplace = True)
data_a.head()

,Date,Timestamp,Value
0,18-May,20200518451,25340
1,18-May,20200518452,25361
2,18-May,20200518453,25365
3,18-May,20200518454,25368
4,18-May,20200518455,25356


In [154]:
data_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       3412 non-null   object
 1   Timestamp  3412 non-null   int64 
 2   Value      3412 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 80.1+ KB


In [155]:
data_a['year']= data_a.Timestamp.apply(lambda x : int(str(x)[:4]))
data_a['month']= data_a.Timestamp.apply(lambda x : int(str(x)[4:6]))
data_a['day']= data_a.Timestamp.apply(lambda x : int(str(x)[6:8]))
data_a.head()

,Date,Timestamp,Value,year,month,day
0,18-May,20200518451,25340,2020,5,18
1,18-May,20200518452,25361,2020,5,18
2,18-May,20200518453,25365,2020,5,18
3,18-May,20200518454,25368,2020,5,18
4,18-May,20200518455,25356,2020,5,18


In [156]:
#Adding time part for 8 days

# i = pd.date_range('2020-05-18', periods=480, freq='3S')
# s = pd.DataFrame(i)
data_timedf = pd.DataFrame()
dates = ['2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24' , '2020-05-25' ]
for i in dates:
  series_ty = pd.date_range(i, periods=480, freq='3S')
  s = pd.DataFrame(series_ty, columns=['time'])
  data_timedf = pd.concat([data_timedf, s])

print(len(data_timedf))
data_timedf.head()


3360


,time
0,2020-05-19 00:00:00
1,2020-05-19 00:00:03
2,2020-05-19 00:00:06
3,2020-05-19 00:00:09
4,2020-05-19 00:00:12


In [157]:
# Time creation for 18- May
i = pd.date_range('2020-05-18', periods=480, freq='3S')
data_timedf_18 = pd.DataFrame(i, columns=['time'])
data_timedf_18 = data_timedf_18.sort_values(['time'], ascending= False)[:30]
data_timedf_18 = data_timedf_18.sort_values(['time'], ascending= True)
print(len(data_timedf_18))
data_timedf_18

# Time creation for 26 - May
i = pd.date_range('2020-05-26', periods=480, freq='3S')
data_timedf_26 = pd.DataFrame(i, columns=['time'])
data_timedf_26 = data_timedf_26.sort_values(['time'], ascending= True)[:22]
# data_timedf_26 = data_timedf_26.sort_values(['time'], ascending= True)[:22]
print(len(data_timedf_26))
data_timedf_26.head()



30
22


,time
0,2020-05-26 00:00:00
1,2020-05-26 00:00:03
2,2020-05-26 00:00:06
3,2020-05-26 00:00:09
4,2020-05-26 00:00:12


In [158]:
## Concatenate all time series 

data_timedf = pd.concat([data_timedf_18, data_timedf, data_timedf_26], axis = 0, sort=True)
data_timedf.sort_values('time', ascending=True)
data_timedf.reset_index(inplace=True)
data_timedf.drop('index', axis = 1, inplace= True)
data_timedf.head()

,time
0,2020-05-18 00:22:30
1,2020-05-18 00:22:33
2,2020-05-18 00:22:36
3,2020-05-18 00:22:39
4,2020-05-18 00:22:42


In [159]:
# Making the complete dataset
data_a = pd.concat([data_a, data_timedf], axis = 1)
data_a.head()

,Date,Timestamp,Value,year,month,day,time
0,18-May,20200518451,25340,2020,5,18,2020-05-18 00:22:30
1,18-May,20200518452,25361,2020,5,18,2020-05-18 00:22:33
2,18-May,20200518453,25365,2020,5,18,2020-05-18 00:22:36
3,18-May,20200518454,25368,2020,5,18,2020-05-18 00:22:39
4,18-May,20200518455,25356,2020,5,18,2020-05-18 00:22:42


In [0]:
data_a.drop(['Date', 'year', 'month', 'day'], axis = 1, inplace = True)
data_a['Value'].astype(int)
data_a.to_csv("data_input.csv", index = False)

# EDA


In [181]:
!pip install auto_ts --no-cache-dir --ignore-installed

     |████████████████████████████████| 51kB 3.7MB/s 
     |██████████▏                     | 2.8MB 6.7MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [178]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import auto_ts as AT 
import pandas as pd

mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['axes.grid'] = False


Running Auto Timeseries version: 0.0.19...Call by using:
        auto_ts.Auto_Timeseries(traindata, ts_column,
                            target, sep,  score_type='rmse', forecast_period=5,
                            time_interval='Month', non_seasonal_pdq=None, seasonality=False,
                            seasonal_period=12, seasonal_PDQ=None, model_type='stats',
                            verbose=1)
    To run three models from Stats, ML and FB Prophet, set model_type='best'
To remove previous versions, perform 'pip uninstall auto_ts'
To get the latest version, perform "pip install auto_ts --no-cache-dir --ignore-installed"


In [0]:

data = pd.read_csv("/content/data_input.csv")
data.drop('Timestamp', axis = 1, inplace = True)
data.head()

In [176]:
trace_1 = go.Scatter(
        x=data.time,
        y= data['Value'],
        name = 'Sensor A',
        line = dict(color = '#29f3c9'),
        opacity = 0.8)

data = [trace_1]
# data = [trace_1]
layout = dict(
        title='Time series for Invalid file 1 and Invalid file 2 ',
        xaxis_title='Time(sec)',
        yaxis_title='Accel norm',
        paper_bgcolor="#dec7f2",
#         paper_bgcolor='rgba(0,1,0,0)'
        )
fig = dict(data=data, layout= layout )
iplot(fig)